# Laboratório 05: Dados Ausentes - Tratamento Univariado (Simples)
**Disciplina:** Extração e Preparação de Dados (IBM8915)
**Objetivo:** Aprender a tomar a decisão crítica entre descartar (`dropna`) ou preencher (`fillna`) buracos nos dados usando medidas de tendência central (Média, Mediana e Moda).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

## Parte 1: Exemplo Guiado - A "Faca" e a "Seringa"
Neste exemplo rápido, usaremos o `.dropna()` (a faca) para amputar o que não tem salvação e o `.fillna()` (a seringa) para curar colunas numéricas e categóricas do nosso DataFrame de pacientes.

In [3]:
# Criando dados fictícios
df_exemplo = pd.DataFrame({
    'Paciente': ['Ana', 'Bruno', 'Carlos', 'Diana', 'Eduardo'],
    'Idade': [25, np.nan, 42, 38, np.nan],         # Numérica
    'Pressao': [120, 125, 130, np.nan, 140],       # Numérica
    'Tipo_Sanguineo': ['O+', 'A-', np.nan, 'O+', np.nan], # Categórica
    'Coluna_Perdida': [np.nan, np.nan, np.nan, np.nan, np.nan] # 100% Nula
})

print("Estado Original:\n", df_exemplo, "\n")

# 1. A Faca: Descartando colunas 100% vazias (axis=1 atinge colunas)
df_exemplo.dropna(axis=1, how='all', inplace=True)

# 2. A Seringa (Numéricos): Preenchendo Idade com a Mediana
mediana_idade = df_exemplo['Idade'].median()
df_exemplo['Idade'].fillna(mediana_idade, inplace=True)

# 3. A Seringa (Categóricos): Preenchendo Tipo Sanguíneo com a Moda
moda_sangue = df_exemplo['Tipo_Sanguineo'].mode()[0] #  pega a primeira moda se houver empate
df_exemplo['Tipo_Sanguineo'].fillna(moda_sangue, inplace=True)

print("Estado Tratado:\n", df_exemplo)

Estado Original:
   Paciente  Idade  Pressao Tipo_Sanguineo  Coluna_Perdida
0      Ana   25.0    120.0             O+             NaN
1    Bruno    NaN    125.0             A-             NaN
2   Carlos   42.0    130.0            NaN             NaN
3    Diana   38.0      NaN             O+             NaN
4  Eduardo    NaN    140.0            NaN             NaN 

Estado Tratado:
   Paciente  Idade  Pressao Tipo_Sanguineo
0      Ana   25.0    120.0             O+
1    Bruno   38.0    125.0             A-
2   Carlos   42.0    130.0             O+
3    Diana   38.0      NaN             O+
4  Eduardo   38.0    140.0             O+


## Parte 2: Exercício Prático (Mão na Massa)
Abaixo geramos um pequeno dataset de imóveis (`df_imoveis`). 
**Sua Tarefa:**
1. Descarte as colunas que possuam **mais de 70%** de valores nulos.
2. Preencha a coluna numérica `Preco` usando a estatística adequada (Média ou Mediana? Dica: Verifique se existem outliers absurdos com `.describe()`).
3. Preencha a coluna categórica `Bairro` com o valor mais comum (Moda).

In [4]:
# NÃO ALTERE ESTE CÓDIGO
np.random.seed(42)
df_imoveis = pd.DataFrame({
    'ID_Imovel': range(1, 101),
    'Bairro': np.random.choice(['Centro', 'Jardins', 'Vila Nova'], 100, p=[0.5, 0.3, 0.2]),
    'Preco': np.append(np.random.normal(300000, 50000, 95), [1500000, 2000000, 2500000, 3000000, 5000000]), # Com outliers milionários
    'Comissao_Corretor_Morta': [np.nan] * 100 # Coluna inútil
})

# Injetando nulos
df_imoveis.loc[np.random.choice(100, 15, replace=False), 'Preco'] = np.nan
df_imoveis.loc[np.random.choice(100, 20, replace=False), 'Bairro'] = np.nan

In [ ]:
# ESCREVA SEU CÓDIGO AQUI
# Passo 1: Use dropna para derrubar a 'Comissao_Corretor_Morta'


# Passo 2: Decida entre .mean() e .median() para 'Preco' e faça o fillna


# Passo 3: Use .mode() para preencher os bairros ausentes



---
## Parte 3: Desafio para Casa - O Pipeline do Engenheiro de Dados
Este desafio consolida as **Aulas 02 a 07**. Rode a célula abaixo para gerar um arquivo CSV no seu disco chamado `ecommerce_messy.csv`.

**A Missão:**
1. **(Aula 02):** Carregue o arquivo lidando com o separador `;` e o encoding `latin-1`.
2. **(Aula 05):** Audite e remova as linhas inteiramente duplicadas.
3. **(Aula 04):** Converta a coluna `Categoria` para o tipo `category` para salvar memória.
4. **(Aula 06):** Plote um `sns.heatmap` para ver onde estão os nulos.
5. **(Aula 07):** Faça a imputação: `Idade` (Numérica) e `Categoria` (Texto).
6. **(Aula 04):** Calcule o Ticket Médio (`Renda`) agrupado por `Categoria`.

In [ ]:
# NÃO ALTERE ESTE CÓDIGO - Geração de dados sujos reais no disco
np.random.seed(10)
n = 800
df_desafio = pd.DataFrame({
    'ID_Cliente': np.random.randint(1000, 9999, n),
    'Idade': np.random.normal(35, 10, n),
    'Renda': np.random.normal(5000, 2000, n),
    'Categoria': np.random.choice(['Eletrônicos', 'Móveis', 'Vestuário'], n)
})

# Inserindo buracos (NaNs)
df_desafio.loc[np.random.choice(n, 120, replace=False), 'Idade'] = np.nan
df_desafio.loc[np.random.choice(n, 60, replace=False), 'Categoria'] = np.nan

# Inserindo linhas duplicadas
df_desafio = pd.concat([df_desafio, df_desafio.sample(45)], ignore_index=True)

# Exportando para o formato legado "Brasileiro"
df_desafio.to_csv('ecommerce_messy.csv', sep=';', encoding='latin-1', index=False)
print("Arquivo 'ecommerce_messy.csv' gerado na sua pasta local!")

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

# 1. Carregue o CSV (read_csv com sep e encoding corretos)

# 2. Remova duplicatas (.drop_duplicates)

# 3. Otimização de tipo (.astype('category'))

# 4. Heatmap do Seaborn (sns.heatmap)

# 5. Tratamento Univariado (fillna com mediana/moda)

# 6. Análise Agrupada (groupby)
